<a href="https://colab.research.google.com/github/bhaveshsingh0206/Contextual-Embedding-based-Stock-Price-Prediction-using-Bidirectional-LSTM/blob/main/Preprocess_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bs4 # in case you don't have it installed
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 106 kB 8.0 MB/s 
     |████████████████████████████████| 287 kB 57.8 MB/s 


In [ ]:
import os
import pandas as pd
from tqdm import tqdm
import numpy as np
import nltk
nltk.download('wordnet')
import re
from bs4 import BeautifulSoup
import contractions
import collections
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/NLP\ Dataset/

/content/drive/.shortcut-targets-by-id/1x6KOC9muicNj4y-nUBUu0S1Oz9Vxunyz/NLP Dataset


## Yahoo Dataset

In [ ]:
!pip install yahooquery

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 46 kB 2.6 MB/s 


In [ ]:
# !pip install yahooquery

from datetime import datetime, timedelta
import requests
from yahooquery import Ticker

def date_range(current_date, num_days=10):
    """Helper function to get x number of days before and after the current date

    Args:
        current_date (str): Current date in YYYY-MM-DD format
        num_days (int): Number of days by which the `current_date` is incremented and decremented

    Returns:
        list: List of dates in the sepcified interval
    """
    num_days = num_days + 1 # Adding 1 because the current date is included in the interval
    base = datetime.strptime(current_date, '%Y-%m-%d')

    
    # Previous `x` days --> 06, 05, 04 (base is 06)
    date_list_prev = [(base - timedelta(days=x)).strftime('%Y-%m-%d') for x in range(num_days)]
    # Next `x` days     --> 06, 07, 08 (base is 06)
    date_list_next = [(base + timedelta(days=x)).strftime('%Y-%m-%d') for x in range(num_days)]
    
    # [06, 05, 04][::-1] + [06, 07, 08][1:]
    # return date_list_prev[::-1] + date_list_next[1:]
    return date_list_prev[::-1]

def get_close_prices(ticker, current_date, num_days=10):
    """Return a list of close prices for every day in the given range of startdate and enddate

    Args:
        ticker (str): Company ticker
        current_date (str): Date (YYYY-MM-DD) for which we need the previous and next 10 close prices
        num_days (str): Number of days offset. Defaults to 10

    Returns:
        list: List of close prices
    """
    tuples = []
    close_prices = []
    first_open = 0

    # Get list of dates for which close price is to be calculated
    dates = date_range(current_date=current_date, num_days=num_days)
    start_date = dates[0]
    end_date = dates[-1]

    # Get close prices
    ticker = ticker.lower()
    ticker = Ticker(ticker, asynchronous=True)
    df = ticker.history(start=start_date, end=end_date)

    # Group data -> [(date, price), (date, price), ...]
    for index, row in df.iterrows():
        # To handle the case where the first price is zero
        # Initialize with open price of the next day
        if first_open == 0:
            first_open = row['open']
        date = index[1].strftime('%Y-%m-%d')
        tuples.append((date, row['close']))
    
    curr_price = first_open
    
    # `dates` is a super set of `close_prices`
    # For every date if there is no close price we take the previus price
    j = 0
    for i in range(len(dates)):
        if j < len(tuples) and tuples[j][0] == dates[i]:
            curr_price = tuples[j][1]
            close_prices.append(curr_price)
            j = j + 1
        else:
            close_prices.append(curr_price)

    return close_prices

def yahoo_data(current_date):
    # Date for which we need the previous and next 10 close prices
    # current_date = '1999-06-06'
    # Number of days by which the `current_date` is incremented and decremented
    num_days = 10
    
    prices = get_close_prices(ticker='^gspc', current_date=current_date, num_days=num_days)
    # print('Length: {}\n\nPrices:\n{}'.format(len(prices), prices))
    return prices


## Create Reuters and Bloomberg CSV

In [ ]:
newdf = pd.DataFrame(columns=['ts', 'title', 'src'])
for filename in tqdm(os.listdir("./reuters 2/")):
  df = pd.read_csv("reuters 2/" + filename, sep='\t', usecols=[0,1])
  df['src'] = 'reuters'
  newdf = newdf.append(df, ignore_index=True)

100%|██████████| 244/244 [00:07<00:00, 34.38it/s]


In [ ]:
newdf

,ts,title,src
0,20070223 11:23 PM EST,Iran vows no weakness over nuclear program,reuters
1,20070223 11:16 PM EST,Citigroup in talks to boost Nikko stake-sources,reuters
2,20070223 11:15 PM EST,U.N. nuclear watchdog head to visit North Korea,reuters
3,20070223 11:15 PM EST,N.Korean nuclear envoy to visit US on March 1:...,reuters
4,20070223 11:08 PM EST,Asylum seekers sent to Australia's Christmas I...,reuters
...,...,...,...
571477,20160805 5:06AM UTC,Fears over Thailand referendum vote,reuters
571478,20160805 4:25AM UTC,"Baseball, surfing among sports approved for To...",reuters
571479,20160805 2:33AM UTC,Trump reacts to Iran alleged cash payment,reuters
571480,20160805 1:52AM UTC,Thailand's army looks to tighten grip on power,reuters


In [ ]:
for idx, v in enumerate(newdf['ts']):
  date = v.split(' ')[0]
  y = date[:4]
  m = date[4:6]
  d = date[6:]
  newdf.at[idx, 'ts'] = y + "-" + m + "-" + d

In [ ]:
newdf

,ts,title,src
0,2007-02-23,Iran vows no weakness over nuclear program,reuters
1,2007-02-23,Citigroup in talks to boost Nikko stake-sources,reuters
2,2007-02-23,U.N. nuclear watchdog head to visit North Korea,reuters
3,2007-02-23,N.Korean nuclear envoy to visit US on March 1:...,reuters
4,2007-02-23,Asylum seekers sent to Australia's Christmas I...,reuters
...,...,...,...
571477,2016-08-05,Fears over Thailand referendum vote,reuters
571478,2016-08-05,"Baseball, surfing among sports approved for To...",reuters
571479,2016-08-05,Trump reacts to Iran alleged cash payment,reuters
571480,2016-08-05,Thailand's army looks to tighten grip on power,reuters


In [ ]:
newdf.to_csv('./reuters.csv')

In [ ]:
# !unzip ./20061020_20131126_bloomberg_news\ 2.zip

In [ ]:
res = []

for folder in tqdm(sorted(os.listdir("./20061020_20131126_bloomberg_news 2/"), reverse=True)):
  for filename in os.listdir("./20061020_20131126_bloomberg_news 2/"+folder):
    file1 = open('./20061020_20131126_bloomberg_news 2/'+folder+'/'+filename, 'r')
    lines = file1.readlines()
    row = []
    for idx, line in enumerate(lines):
      if idx == 0:
        row.append(line[3:-1])
      if idx == 2:
        row.append(line.split('T')[0][3:])
        break
    row = row[::-1]
    res.append(row)

100%|██████████| 1944/1944 [02:23<00:00, 13.54it/s] 


In [ ]:
len(res)

83924

In [ ]:
bloomberg_df = pd.DataFrame(res, columns = ['ts', 'title', 'src'])

In [ ]:
bloomberg_df

,ts,title,src
0,2013-11-08,Mexico Peso Falls as U.S. Payrolls Add to Fed ...,bloomberg
1,2013-11-10,Republicans Take Obamacare Strategy to War on ...,bloomberg
2,2013-11-08,"SAC, Prosecutors Oppose Wyeth-Elan Investor Re...",bloomberg
3,2013-11-08,U.K. Stocks Are Little Changed as U.S. Payroll...,bloomberg
4,2013-11-08,No Fracking in Germany for Now Backed in Merke...,bloomberg
...,...,...,...
83919,2006-10-27,Singapore Airlines 2nd-Quarter Net Falls 15% o...,bloomberg
83920,2006-10-27,Baoshan's 3rd-Quarter Profit Gains on Steel De...,bloomberg
83921,2006-10-27,MPC Third-Quarter Net Doubles on U.K. Property...,bloomberg
83922,2006-10-21,"Jim Cramer: Diageo, Anheuser-Busch, Monster Wo...",bloomberg


In [ ]:
bloomberg_df.to_csv('./bloomberg.csv')

## Merge Data

In [ ]:
bloomberg_df = pd.read_csv('./bloomberg.csv', index_col=0)
bloomberg_df

,ts,title,src
0,2013-11-08,Mexico Peso Falls as U.S. Payrolls Add to Fed ...,bloomberg
1,2013-11-10,Republicans Take Obamacare Strategy to War on ...,bloomberg
2,2013-11-08,"SAC, Prosecutors Oppose Wyeth-Elan Investor Re...",bloomberg
3,2013-11-08,U.K. Stocks Are Little Changed as U.S. Payroll...,bloomberg
4,2013-11-08,No Fracking in Germany for Now Backed in Merke...,bloomberg
...,...,...,...
83919,2006-10-27,Singapore Airlines 2nd-Quarter Net Falls 15% o...,bloomberg
83920,2006-10-27,Baoshan's 3rd-Quarter Profit Gains on Steel De...,bloomberg
83921,2006-10-27,MPC Third-Quarter Net Doubles on U.K. Property...,bloomberg
83922,2006-10-21,"Jim Cramer: Diageo, Anheuser-Busch, Monster Wo...",bloomberg


In [ ]:
reuters_df = pd.read_csv('./reuters.csv', index_col=0)
reuters_df

,ts,title,src
0,2007-02-23,Iran vows no weakness over nuclear program,reuters
1,2007-02-23,Citigroup in talks to boost Nikko stake-sources,reuters
2,2007-02-23,U.N. nuclear watchdog head to visit North Korea,reuters
3,2007-02-23,N.Korean nuclear envoy to visit US on March 1:...,reuters
4,2007-02-23,Asylum seekers sent to Australia's Christmas I...,reuters
...,...,...,...
571477,2016-08-05,Fears over Thailand referendum vote,reuters
571478,2016-08-05,"Baseball, surfing among sports approved for To...",reuters
571479,2016-08-05,Trump reacts to Iran alleged cash payment,reuters
571480,2016-08-05,Thailand's army looks to tighten grip on power,reuters


In [ ]:
merged_df = pd.DataFrame(columns=['ts', 'title', 'src'])
merged_df = merged_df.append(bloomberg_df, ignore_index=True)
merged_df = merged_df.append(reuters_df, ignore_index=True)
merged_df

,ts,title,src
0,2013-11-08,Mexico Peso Falls as U.S. Payrolls Add to Fed ...,bloomberg
1,2013-11-10,Republicans Take Obamacare Strategy to War on ...,bloomberg
2,2013-11-08,"SAC, Prosecutors Oppose Wyeth-Elan Investor Re...",bloomberg
3,2013-11-08,U.K. Stocks Are Little Changed as U.S. Payroll...,bloomberg
4,2013-11-08,No Fracking in Germany for Now Backed in Merke...,bloomberg
...,...,...,...
655401,2016-08-05,Fears over Thailand referendum vote,reuters
655402,2016-08-05,"Baseball, surfing among sports approved for To...",reuters
655403,2016-08-05,Trump reacts to Iran alleged cash payment,reuters
655404,2016-08-05,Thailand's army looks to tighten grip on power,reuters


In [ ]:
merged_df.shape

(655406, 3)

In [ ]:
merged_df.to_csv('../merged.csv')

## Cleaning Data

In [ ]:
bloomberg_df = pd.read_csv('./bloomberg.csv', index_col=0)
bloomberg_df

,ts,title,src
0,2013-11-08,Mexico Peso Falls as U.S. Payrolls Add to Fed ...,bloomberg
1,2013-11-10,Republicans Take Obamacare Strategy to War on ...,bloomberg
2,2013-11-08,"SAC, Prosecutors Oppose Wyeth-Elan Investor Re...",bloomberg
3,2013-11-08,U.K. Stocks Are Little Changed as U.S. Payroll...,bloomberg
4,2013-11-08,No Fracking in Germany for Now Backed in Merke...,bloomberg
...,...,...,...
83919,2006-10-27,Singapore Airlines 2nd-Quarter Net Falls 15% o...,bloomberg
83920,2006-10-27,Baoshan's 3rd-Quarter Profit Gains on Steel De...,bloomberg
83921,2006-10-27,MPC Third-Quarter Net Doubles on U.K. Property...,bloomberg
83922,2006-10-21,"Jim Cramer: Diageo, Anheuser-Busch, Monster Wo...",bloomberg


In [ ]:
reuters_df = pd.read_csv('./reuters.csv', index_col=0)
reuters_df

,ts,title,src
0,2007-02-23,Iran vows no weakness over nuclear program,reuters
1,2007-02-23,Citigroup in talks to boost Nikko stake-sources,reuters
2,2007-02-23,U.N. nuclear watchdog head to visit North Korea,reuters
3,2007-02-23,N.Korean nuclear envoy to visit US on March 1:...,reuters
4,2007-02-23,Asylum seekers sent to Australia's Christmas I...,reuters
...,...,...,...
571477,2016-08-05,Fears over Thailand referendum vote,reuters
571478,2016-08-05,"Baseball, surfing among sports approved for To...",reuters
571479,2016-08-05,Trump reacts to Iran alleged cash payment,reuters
571480,2016-08-05,Thailand's army looks to tighten grip on power,reuters


In [ ]:
merged_df = pd.read_csv('./merged.csv', index_col=0)
merged_df

,ts,title,src
0,2013-11-08,Mexico Peso Falls as U.S. Payrolls Add to Fed ...,bloomberg
1,2013-11-10,Republicans Take Obamacare Strategy to War on ...,bloomberg
2,2013-11-08,"SAC, Prosecutors Oppose Wyeth-Elan Investor Re...",bloomberg
3,2013-11-08,U.K. Stocks Are Little Changed as U.S. Payroll...,bloomberg
4,2013-11-08,No Fracking in Germany for Now Backed in Merke...,bloomberg
...,...,...,...
655401,2016-08-05,Fears over Thailand referendum vote,reuters
655402,2016-08-05,"Baseball, surfing among sports approved for To...",reuters
655403,2016-08-05,Trump reacts to Iran alleged cash payment,reuters
655404,2016-08-05,Thailand's army looks to tighten grip on power,reuters


In [ ]:
def calculateAverage(df, reviewBody, cleanBody):
  originalUncleanedAverage=df[reviewBody].apply(lambda x: len(x)).mean()
  originalCleanedAverage=df[cleanBody].apply(lambda x: len(x)).mean()
  print(f'Original uncleaned reviews had an average length of ${originalUncleanedAverage}')
  print(f'New cleaned reviews had an average length of ${originalCleanedAverage}')

In [ ]:
punctuations = '''!()-[]{};:"\,<>./?@#%^&*_~'''
def cleanText(x):
  x = x.lower().strip()
  soup = BeautifulSoup(x)
  x = soup.get_text()
  x = re.sub(r'https?://\S+', '', x)
  x=re.sub("\s\s+", " ", x.strip())
  # x=re.sub("[^a-zA-Z\s]+", " ", x)

  no_punct = ""
  for char in x:
    if char not in punctuations:
        no_punct = no_punct + char
    else:
        no_punct += " "
  x = no_punct

  x=re.sub("\s\s+", " ", x.strip())
  if (x==' ' or len(x)==0): 
    return np.nan
  return x

In [ ]:
reuters_df['clean_title'] = reuters_df['title'].apply(cleanText)

In [ ]:
reuters_df.dropna(inplace=True)
reuters_df.reset_index(inplace=True, drop=True)

In [ ]:
reuters_df

,ts,title,src,clean_title
0,2007-02-23,Iran vows no weakness over nuclear program,reuters,iran vows no weakness over nuclear program
1,2007-02-23,Citigroup in talks to boost Nikko stake-sources,reuters,citigroup in talks to boost nikko stake sources
2,2007-02-23,U.N. nuclear watchdog head to visit North Korea,reuters,u n nuclear watchdog head to visit north korea
3,2007-02-23,N.Korean nuclear envoy to visit US on March 1:...,reuters,n korean nuclear envoy to visit us on march 1 ...
4,2007-02-23,Asylum seekers sent to Australia's Christmas I...,reuters,asylum seekers sent to australia's christmas i...
...,...,...,...,...
571467,2016-08-05,Fears over Thailand referendum vote,reuters,fears over thailand referendum vote
571468,2016-08-05,"Baseball, surfing among sports approved for To...",reuters,baseball surfing among sports approved for tok...
571469,2016-08-05,Trump reacts to Iran alleged cash payment,reuters,trump reacts to iran alleged cash payment
571470,2016-08-05,Thailand's army looks to tighten grip on power,reuters,thailand's army looks to tighten grip on power


In [ ]:
calculateAverage(reuters_df, 'title', 'clean_title')

Original uncleaned reviews had an average length of $65.83422634879749
New cleaned reviews had an average length of $64.53181258224375


In [ ]:
bloomberg_df.dropna(inplace=True)

In [ ]:
bloomberg_df['clean_title'] = bloomberg_df['title'].apply(cleanText)
bloomberg_df.dropna(inplace=True)
bloomberg_df.reset_index(inplace=True)
bloomberg_df

,index,ts,title,src,clean_title
0,0,2013-11-08,Mexico Peso Falls as U.S. Payrolls Add to Fed ...,bloomberg,mexico peso falls as u s payrolls add to fed t...
1,1,2013-11-10,Republicans Take Obamacare Strategy to War on ...,bloomberg,republicans take obamacare strategy to war on ...
2,2,2013-11-08,"SAC, Prosecutors Oppose Wyeth-Elan Investor Re...",bloomberg,sac prosecutors oppose wyeth elan investor req...
3,3,2013-11-08,U.K. Stocks Are Little Changed as U.S. Payroll...,bloomberg,u k stocks are little changed as u s payrolls ...
4,4,2013-11-08,No Fracking in Germany for Now Backed in Merke...,bloomberg,no fracking in germany for now backed in merke...
...,...,...,...,...,...
58299,58299,2006-10-27,Singapore Airlines 2nd-Quarter Net Falls 15% o...,bloomberg,singapore airlines 2nd quarter net falls 15 on...
58300,58300,2006-10-27,Baoshan's 3rd-Quarter Profit Gains on Steel De...,bloomberg,baoshan's 3rd quarter profit gains on steel de...
58301,58301,2006-10-27,MPC Third-Quarter Net Doubles on U.K. Property...,bloomberg,mpc third quarter net doubles on u k property ...
58302,58302,2006-10-21,"Jim Cramer: Diageo, Anheuser-Busch, Monster Wo...",bloomberg,jim cramer diageo anheuser busch monster world...


In [ ]:
calculateAverage(bloomberg_df, 'title', 'clean_title')

Original uncleaned reviews had an average length of $59.9691787870472
New cleaned reviews had an average length of $59.283376783754115


In [ ]:
bloomberg_df = bloomberg_df.drop('index', axis=1)

In [ ]:
bloomberg_df

,ts,title,src,clean_title
0,2013-11-08,Mexico Peso Falls as U.S. Payrolls Add to Fed ...,bloomberg,mexico peso falls as u s payrolls add to fed t...
1,2013-11-10,Republicans Take Obamacare Strategy to War on ...,bloomberg,republicans take obamacare strategy to war on ...
2,2013-11-08,"SAC, Prosecutors Oppose Wyeth-Elan Investor Re...",bloomberg,sac prosecutors oppose wyeth elan investor req...
3,2013-11-08,U.K. Stocks Are Little Changed as U.S. Payroll...,bloomberg,u k stocks are little changed as u s payrolls ...
4,2013-11-08,No Fracking in Germany for Now Backed in Merke...,bloomberg,no fracking in germany for now backed in merke...
...,...,...,...,...
58299,2006-10-27,Singapore Airlines 2nd-Quarter Net Falls 15% o...,bloomberg,singapore airlines 2nd quarter net falls 15 on...
58300,2006-10-27,Baoshan's 3rd-Quarter Profit Gains on Steel De...,bloomberg,baoshan's 3rd quarter profit gains on steel de...
58301,2006-10-27,MPC Third-Quarter Net Doubles on U.K. Property...,bloomberg,mpc third quarter net doubles on u k property ...
58302,2006-10-21,"Jim Cramer: Diageo, Anheuser-Busch, Monster Wo...",bloomberg,jim cramer diageo anheuser busch monster world...


In [ ]:
reuters_df.to_csv('./reuters_clean.csv')
bloomberg_df.to_csv('./bloomberg_clean.csv')

In [ ]:
merged_df = pd.DataFrame(columns=['ts', 'title', 'src', 'clean_title'])
merged_df = merged_df.append(bloomberg_df, ignore_index=True)
merged_df = merged_df.append(reuters_df, ignore_index=True)
merged_df

,ts,title,src,clean_title
0,2013-11-08,Mexico Peso Falls as U.S. Payrolls Add to Fed ...,bloomberg,mexico peso falls as u s payrolls add to fed t...
1,2013-11-10,Republicans Take Obamacare Strategy to War on ...,bloomberg,republicans take obamacare strategy to war on ...
2,2013-11-08,"SAC, Prosecutors Oppose Wyeth-Elan Investor Re...",bloomberg,sac prosecutors oppose wyeth elan investor req...
3,2013-11-08,U.K. Stocks Are Little Changed as U.S. Payroll...,bloomberg,u k stocks are little changed as u s payrolls ...
4,2013-11-08,No Fracking in Germany for Now Backed in Merke...,bloomberg,no fracking in germany for now backed in merke...
...,...,...,...,...
629771,2016-08-05,Fears over Thailand referendum vote,reuters,fears over thailand referendum vote
629772,2016-08-05,"Baseball, surfing among sports approved for To...",reuters,baseball surfing among sports approved for tok...
629773,2016-08-05,Trump reacts to Iran alleged cash payment,reuters,trump reacts to iran alleged cash payment
629774,2016-08-05,Thailand's army looks to tighten grip on power,reuters,thailand's army looks to tighten grip on power


In [ ]:
merged_df.to_csv('./merged_clean.csv')

## Creating final data with Yahoo values

In [ ]:
merged_df = pd.read_csv('./merged_clean.csv', index_col=0)
merged_df

,ts,title,src,clean_title
0,2013-11-08,Mexico Peso Falls as U.S. Payrolls Add to Fed ...,bloomberg,mexico peso falls as u s payrolls add to fed t...
1,2013-11-10,Republicans Take Obamacare Strategy to War on ...,bloomberg,republicans take obamacare strategy to war on ...
2,2013-11-08,"SAC, Prosecutors Oppose Wyeth-Elan Investor Re...",bloomberg,sac prosecutors oppose wyeth elan investor req...
3,2013-11-08,U.K. Stocks Are Little Changed as U.S. Payroll...,bloomberg,u k stocks are little changed as u s payrolls ...
4,2013-11-08,No Fracking in Germany for Now Backed in Merke...,bloomberg,no fracking in germany for now backed in merke...
...,...,...,...,...
629771,2016-08-05,Fears over Thailand referendum vote,reuters,fears over thailand referendum vote
629772,2016-08-05,"Baseball, surfing among sports approved for To...",reuters,baseball surfing among sports approved for tok...
629773,2016-08-05,Trump reacts to Iran alleged cash payment,reuters,trump reacts to iran alleged cash payment
629774,2016-08-05,Thailand's army looks to tighten grip on power,reuters,thailand's army looks to tighten grip on power


In [ ]:
df2 = merged_df.sort_values('ts', ascending=False, ignore_index=True)

In [ ]:
df2

,ts,title,src,clean_title
0,INVA-LI-D_DATE,<strong>reached an agreement with GM</strong>,reuters,reached an agreement with gm
1,INVA-LI-D_DATE,Effective Control Transport Inc. Article in C...,reuters,effective control transport inc article in can...
2,B y P h i l i p S c r a n t o n\n,When Hitler Took,bloomberg,when hitler took
3,2016-08-11,"UPDATE 2-Brazil's Senate indicts Rousseff, ope...",reuters,update 2 brazil's senate indicts rousseff open...
4,2016-08-11,Argentina foreign investment quest a work in p...,reuters,argentina foreign investment quest a work in p...
...,...,...,...,...
629771,2006-10-27,Singapore Airlines 2nd-Quarter Net Falls 15% o...,bloomberg,singapore airlines 2nd quarter net falls 15 on...
629772,2006-10-27,Baoshan's 3rd-Quarter Profit Gains on Steel De...,bloomberg,baoshan's 3rd quarter profit gains on steel de...
629773,2006-10-27,MPC Third-Quarter Net Doubles on U.K. Property...,bloomberg,mpc third quarter net doubles on u k property ...
629774,2006-10-21,"Jim Cramer: Diageo, Anheuser-Busch, Monster Wo...",bloomberg,jim cramer diageo anheuser busch monster world...


In [ ]:
df2=df2.drop(df2.index[[0,1,2]])
df2.reset_index(inplace=True)

In [ ]:
df2 = df2.drop('index', axis=1)
df2

,ts,title,src,clean_title
0,2016-08-11,"UPDATE 2-Brazil's Senate indicts Rousseff, ope...",reuters,update 2 brazil's senate indicts rousseff open...
1,2016-08-11,Argentina foreign investment quest a work in p...,reuters,argentina foreign investment quest a work in p...
2,2016-08-11,UPDATE 2-Two planes land safely at Brussels ai...,reuters,update 2 two planes land safely at brussels ai...
3,2016-08-11,BRIEF-Pershing Square Holdings - NAV per share...,reuters,brief pershing square holdings nav per share a...
4,2016-08-11,U.S. says it's aware of reports Vietnam fortif...,reuters,u s says it's aware of reports vietnam fortifi...
...,...,...,...,...
629768,2006-10-27,Singapore Airlines 2nd-Quarter Net Falls 15% o...,bloomberg,singapore airlines 2nd quarter net falls 15 on...
629769,2006-10-27,Baoshan's 3rd-Quarter Profit Gains on Steel De...,bloomberg,baoshan's 3rd quarter profit gains on steel de...
629770,2006-10-27,MPC Third-Quarter Net Doubles on U.K. Property...,bloomberg,mpc third quarter net doubles on u k property ...
629771,2006-10-21,"Jim Cramer: Diageo, Anheuser-Busch, Monster Wo...",bloomberg,jim cramer diageo anheuser busch monster world...


In [ ]:
df2.to_csv('./merged_clean_sorted.csv')

In [ ]:
df2 = pd.read_csv('./merged_clean_sorted.csv', index_col=0)
df2

,ts,title,src,clean_title
0,2016-08-11,"UPDATE 2-Brazil's Senate indicts Rousseff, ope...",reuters,update 2 brazil's senate indicts rousseff open...
1,2016-08-11,Argentina foreign investment quest a work in p...,reuters,argentina foreign investment quest a work in p...
2,2016-08-11,UPDATE 2-Two planes land safely at Brussels ai...,reuters,update 2 two planes land safely at brussels ai...
3,2016-08-11,BRIEF-Pershing Square Holdings - NAV per share...,reuters,brief pershing square holdings nav per share a...
4,2016-08-11,U.S. says it's aware of reports Vietnam fortif...,reuters,u s says it's aware of reports vietnam fortifi...
...,...,...,...,...
629768,2006-10-27,Singapore Airlines 2nd-Quarter Net Falls 15% o...,bloomberg,singapore airlines 2nd quarter net falls 15 on...
629769,2006-10-27,Baoshan's 3rd-Quarter Profit Gains on Steel De...,bloomberg,baoshan's 3rd quarter profit gains on steel de...
629770,2006-10-27,MPC Third-Quarter Net Doubles on U.K. Property...,bloomberg,mpc third quarter net doubles on u k property ...
629771,2006-10-21,"Jim Cramer: Diageo, Anheuser-Busch, Monster Wo...",bloomberg,jim cramer diageo anheuser busch monster world...


In [ ]:
df2['ts'].value_counts().sort_values(ascending=True) 

2006-10-20       1
2013-05-24       1
2013-05-14       1
2011-12-23       1
2012-02-09       1
              ... 
2011-10-25    5839
2012-12-18    5999
2012-07-31    6299
2011-05-03    6622
2011-04-28    6788
Name: ts, Length: 708, dtype: int64

In [ ]:
len(df2['ts'].value_counts())

708

In [ ]:
m = collections.defaultdict(list)
cnt = collections.defaultdict(lambda: 0)

for index, row in tqdm(df2.iterrows()):
    if cnt[row['ts']]>400:
      continue
    else:
      if row['ts'] in m:
        for idx, i in enumerate(m[row['ts']]):
          df2.loc[index,'p'+str(idx)] = i
      else:
        yd = yahoo_data(row['ts'])
        for idx, i in enumerate(yd):
          df2.loc[index,'p'+str(idx)] = i
          m[row['ts']] = yd
      cnt[row['ts']] += 1

324933it [2:38:41, 17.26it/s]

In [ ]:
df2

In [ ]:
m = collections.defaultdict(list)
cnt = collections.defaultdict(lambda: 0)

for index, row in tqdm(df2.iterrows()):
    if cnt[row['ts']]>400:
      continue
    else:
      if row['ts'] in m:
        for idx, i in enumerate(m[row['ts']]):
          df2.loc[index,'p'+str(idx)] = i
      else:
        yd = yahoo_data(row['ts'])
        for idx, i in enumerate(yd):
          df2.loc[index,'p'+str(idx)] = i
          m[row['ts']] = yd
      cnt[row['ts']] += 1

199it [01:02,  3.17it/s]


KeyboardInterrupt: ignored

In [ ]:
df2 = df2.dropna()
df2.reset_index(inplace=True)

In [ ]:
df2 = df2.drop('index', axis=1)
df2

,ts,title,src,clean_title,yahoo
0,2016-08-11,"UPDATE 2-Brazil's Senate indicts Rousseff, ope...",reuters,update 2 brazil's senate indicts rousseff open...,"[2170.840087890625, 2157.030029296875, 2163.79..."
1,2016-08-11,Argentina foreign investment quest a work in p...,reuters,argentina foreign investment quest a work in p...,"[2170.840087890625, 2157.030029296875, 2163.79..."
2,2016-08-11,UPDATE 2-Two planes land safely at Brussels ai...,reuters,update 2 two planes land safely at brussels ai...,"[2170.840087890625, 2157.030029296875, 2163.79..."
3,2016-08-11,BRIEF-Pershing Square Holdings - NAV per share...,reuters,brief pershing square holdings nav per share a...,"[2170.840087890625, 2157.030029296875, 2163.79..."
4,2016-08-11,U.S. says it's aware of reports Vietnam fortif...,reuters,u s says it's aware of reports vietnam fortifi...,"[2170.840087890625, 2157.030029296875, 2163.79..."
...,...,...,...,...,...
144400,2006-10-27,Singapore Airlines 2nd-Quarter Net Falls 15% o...,bloomberg,singapore airlines 2nd quarter net falls 15 on...,"[1364.050048828125, 1365.800048828125, 1366.95..."
144401,2006-10-27,Baoshan's 3rd-Quarter Profit Gains on Steel De...,bloomberg,baoshan's 3rd quarter profit gains on steel de...,"[1364.050048828125, 1365.800048828125, 1366.95..."
144402,2006-10-27,MPC Third-Quarter Net Doubles on U.K. Property...,bloomberg,mpc third quarter net doubles on u k property ...,"[1364.050048828125, 1365.800048828125, 1366.95..."
144403,2006-10-21,"Jim Cramer: Diageo, Anheuser-Busch, Monster Wo...",bloomberg,jim cramer diageo anheuser busch monster world...,"[1349.949951171875, 1362.8299560546875, 1365.6..."


In [ ]:
df2.to_csv('./final_data.csv')